In [1]:
# Imports
import datetime
import warnings
import pandas as pd
from scripts.main import laad_data, bekijk_ziekteverzuim, bekijk_clienten, bekijk_flexpool, kies_onderwerp, pas_voortschrijdend_gemiddelde_toe, pas_regressie_toe, onderzoek_afwijkingen, bereken_metrieken, voorspel, optie_1, optie_2, optie_3

## Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)
datum_vandaag = datetime.datetime.now().strftime("%Y-%m-%d")
# Disable warnings
warnings.filterwarnings('ignore')

In [2]:
data = laad_data()
# onderwerp = kies_onderwerp()

In [3]:
data.head()

,Cliënten,Ziekteverzuim,Flexpool
Datum,,,
2019-01-01,239,8.670,8
2019-01-02,239,8.590,9
2019-01-03,239,8.740,10
2019-01-04,238,8.900,9
2019-01-05,237,8.740,9


In [4]:
df_metrics_vg = pd.DataFrame()
for onderwerp in ['Ziekteverzuim']:
    for vensterlengte in [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 73, 365]:
        for verschuiving in [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 73, 365, 730]:
            voortschrijdend_gemiddelde = pas_voortschrijdend_gemiddelde_toe(data=data, 
                                                                            onderwerp=onderwerp, 
                                                                            vensterlengte=vensterlengte, 
                                                                            verschuiving=verschuiving, 
                                                                            plot=False)
            metrieken = bereken_metrieken(list_of_dfs=[voortschrijdend_gemiddelde], 
                                        onderwerp=onderwerp,
                                        print_statement=False)
            metrieken['onderwerp'] = onderwerp
            metrieken['vensterlengte'] = vensterlengte
            metrieken['verschuiving'] = verschuiving
            df_metrics_vg = pd.concat([df_metrics_vg, metrieken])

In [ ]:
regressie = pas_regressie_toe(data=data, onderwerp=onderwerp)

df_metrics_lr = pd.DataFrame()
for onderwerp in ['Ziekteverzuim']:
    for jaarlijks_seizoenspatroon in [True, False]:
        for wekelijks_seizoenspatroon in [True, False]:
            for graad in range(1,24):
                regressie = pas_regressie_toe(data=data, 
                                            onderwerp=onderwerp,
                                            jaarlijks_seizoenspatroon=jaarlijks_seizoenspatroon,
                                            wekelijks_seizoenspatroon=wekelijks_seizoenspatroon,
                                            graad=graad,
                                            plot=False)
                metrieken = bereken_metrieken(list_of_dfs=[regressie], 
                                            onderwerp=onderwerp,
                                            print_statement=False)
                metrieken['onderwerp'] = onderwerp
                metrieken['jaarlijks_seizoenspatroon'] = jaarlijks_seizoenspatroon
                metrieken['wekelijks_seizoenspatroon'] = wekelijks_seizoenspatroon
                metrieken['graad'] = graad
                df_metrics_lr = pd.concat([df_metrics_lr, metrieken])

In [ ]:
df_metrics_vg.to_csv(f"data/voortschrijdend_gemiddelde_{onderwerp}_{datum_vandaag}.csv", index=False)
df_metrics_lr.to_csv(f"data/regressie_{onderwerp}_{datum_vandaag}.csv", index=False)